## Semantic Search On Turkish News Data

In this notebook, We are going to create simple semantic search for Turkish news dataset.
We are going to download dataset from kaggle and encode it using bert model's turkish version.


In [ ]:
!pip install qdrant-client transformers
!pip install sentence_transformers
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- Lets first get the data from kaggle
- Our data is a news data with thier class
- But our case will be using it for similarity search
- You can obtain your kaggle credentials from your kaggle profile

In [ ]:
import json
import zipfile
import numpy as np
import os
import pandas as pd

In [ ]:
kaggle_api_key = os.environ['KAGGLE_API_KEY']
kaggle_username = os.environ['KAGGLE_USERNAME']

In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

### get your kaggle token from kaggle.com
api_token = {"username":kaggle_username,"key":kaggle_api_key}


with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download savasy/ttc4900

 60% 2.00M/3.34M [00:00<00:00, 2.96MB/s]
100% 3.34M/3.34M [00:00<00:00, 3.91MB/s]


In [ ]:
dataset_zip = "/content/ttc4900.zip"
with zipfile.ZipFile(dataset_zip, "r") as zip_ref:
    zip_ref.extractall("/content")

In [ ]:
df = pd.read_csv('/content/7allV03.csv')
df

,category,text
0,siyaset,3 milyon ile ön seçim vaadi mhp nin 10 olağan...
1,siyaset,mesut_yılmaz yüce_divan da ceza alabilirdi pr...
2,siyaset,disko lar kaldırılıyor başbakan_yardımcısı ar...
3,siyaset,sarıgül anayasa_mahkemesi ne gidiyor mustafa_...
4,siyaset,erdoğan idamın bir haklılık sebebi var demek ...
...,...,...
4895,teknoloji,iphone lara geri dönüyor ios 6 sürümüyle tele...
4896,teknoloji,muslukta devrim sadece elimizi yıkadığımız mu...
4897,teknoloji,halka iyi anlatılmalı bilgi_teknolojileri ile...
4898,teknoloji,çöpe gidiyorlar apple 775 bin uygulamayla app...


import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df, *['category'], **{})
chart

- Since Dataset is in Turkish Language, we are going to use an embedding model for Turkish language.

In [ ]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("emrecan/bert-base-turkish-cased-mean-nli-stsb-tr")

In [ ]:
from qdrant_client import QdrantClient

- Before connecting, make sure you have  account [Qdrant](https://qdrant.tech/)
- In your account you can go to cloud and get your credentials for connecting

In [ ]:
collection_name = "news_reccomender"
qdrant_api_key = os.environ['QDRANT_API_KEY']
qdrant_url = os.environ['QDRANT_URL']

In [ ]:
qdrant_client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_api_key
     )

Creating a collection is a simple task, here we name our collection news_reccomender

In [ ]:
qdrant_client.recreate_collection(
	collection_name=collection_name,
	vectors_config=models.VectorParams(
		size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
		distance=models.Distance.COSINE,
		on_disk=True
	)
)

True

- Once you created you collection, you can use following code to upload your records
- You can also verify your collection with followng code

In [ ]:
collection_info = qdrant_client.get_collection(collection_name=collection_name)
list(collection_info)

[('status', <CollectionStatus.GREEN: 'green'>),
 ('optimizer_status', <OptimizersStatusOneOf.OK: 'ok'>),
 ('vectors_count', 0),
 ('indexed_vectors_count', 0),
 ('points_count', 0),
 ('segments_count', 2),
 ('config',
  CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=True), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None)),
 ('payload_schema', {})]

In [ ]:
qdrant_client.upload_records(
	collection_name=collection_name,
	records=[
		models.Record(
			id=idx,
			vector=encoder.encode(doc["text"]).tolist(),
			payload=doc
		) for idx, doc in enumerate(df.to_dict(orient='records'))
	]
)

In [ ]:
collection_info = qdrant_client.get_collection(collection_name=collection_name)
list(collection_info)

[('status', <CollectionStatus.GREEN: 'green'>),
 ('optimizer_status', <OptimizersStatusOneOf.OK: 'ok'>),
 ('vectors_count', 4900),
 ('indexed_vectors_count', 0),
 ('points_count', 4900),
 ('segments_count', 2),
 ('config',
  CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=True), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None)),
 ('payload_schema', {})]

Now lets recheck our collection

In [ ]:
hits = qdrant_client.search(
	collection_name=collection_name,
	query_vector=encoder.encode("Cumhurbaskani yurtdisina").tolist(),
	limit=3
)
for hit in hits:
	print(hit.payload, "score:", hit.score)

{'category': 'ekonomi ', 'text': ' erdoğan dünyayı dört kez turladı başbakan recep_tayyip_erdoğan 2012 \x92 de 22 ülkeye 24 ziyaret gerçekleştirdi başbakan_erdoğan \x92 ın 2012 \x92 de ziyaret ettiği ülkeler şöyle güney_kore çin azerbaycan 2 pakistan 2 meksika brezilya endonezya brunei bosna_hersek ukrayna slovenya almanya ispanya italya ingiltere bulgaristan katar mısır suudi_arabistan rusya kazakistan ve iran yurt dışı temasları için toplam 165 bin 120 kilometre yol kateden erdoğan \x92 ın en uzun yolculuğu g 20 zirvesi \x92 ne katılmak için 12 bin kilometre yol katederek ulaştığı meksika \x92 nın los cabos kentine oldu erdoğan \x92 ın yurt dışı ziyaretlerinde katettiği toplam 165 bin 120 kilometre dünya çevresinde yapılacak 4 tura denk geliyor'} score: 0.43037367
{'category': 'siyaset ', 'text': ' emniyet_müdürü nü hemen görevden alın mhp_grup_başkanvekili_oktay_vural diyarbakır emniyet_müdürü recep güven in hemen görevden alınması gerektiğini iddia etti tbmm de basın toplantısı düz